# Kafka Real-time Consumer

Simulation of real-time data consumer from a Kafka message queue.

---

### 1. Import the required libraries

In [1]:
import $ivy.`org.apache.kafka:kafka-clients:3.4.0`
import $ivy.`org.scalanlp::breeze-viz:2.1.0`
import $ivy.`log4j:log4j:1.2.17`
//import $ivy.`org.slf4j:slf4j-jdk14:2.0.7`

import $ivy.$                                     

import $ivy.$                               

import $ivy.$                   
//import $ivy.`org.slf4j:slf4j-jdk14:2.0.7`

In [2]:
import org.apache.kafka.common.serialization.{ByteArrayDeserializer, ByteArraySerializer,
                                              LongDeserializer, LongSerializer, StringDeserializer, StringSerializer}
import org.apache.kafka.clients.consumer.{ConsumerConfig, KafkaConsumer}
import scala.jdk.CollectionConverters._

import org.apache.kafka.common.serialization.{ByteArrayDeserializer, ByteArraySerializer,
                                              LongDeserializer, LongSerializer, StringDeserializer, StringSerializer}

import org.apache.kafka.clients.consumer.{ConsumerConfig, KafkaConsumer}

import scala.jdk.CollectionConverters._

In [3]:
import scala.io.Source
import scala.util.control.Breaks._
import java.io.File
import java.util.Properties
import java.nio.ByteBuffer
import java.sql.Timestamp

import scala.io.Source

import scala.util.control.Breaks._

import java.io.File

import java.util.Properties

import java.nio.ByteBuffer

import java.sql.Timestamp

In [4]:
import breeze.linalg._
import breeze.plot._
import org.apache.log4j.{Level, Logger}

import breeze.linalg._

import breeze.plot._

import org.apache.log4j.{Level, Logger}

## 2. Initialise the Environment


In [5]:
val logger: Logger = Logger.getLogger("Kafka_Realtime_Sink")
logger.setLevel(Level.INFO)

logger: Logger = org.apache.log4j.Logger@277d2440

In [6]:
var topic:String = "credit_card_txns";
var broker:String = "localhost:9092";
val groupId = "group1"

topic: String = "credit_card_txns"
broker: String = "localhost:9092"
groupId: String = "group1"

### 2.1 Configure the Consumer

In [7]:
val props = new Properties()
props.put(ConsumerConfig.BOOTSTRAP_SERVERS_CONFIG, broker)
props.put(ConsumerConfig.KEY_DESERIALIZER_CLASS_CONFIG, classOf[LongDeserializer].getName)
props.put(ConsumerConfig.VALUE_DESERIALIZER_CLASS_CONFIG, classOf[ByteArrayDeserializer].getName)
props.put(ConsumerConfig.GROUP_ID_CONFIG, groupId)

val consumer = new KafkaConsumer[Long, Array[Byte]](props)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


props: Properties = {key.deserializer=org.apache.kafka.common.serialization.LongDeserializer, value.deserializer=org.apache.kafka.common.serialization.ByteArrayDeserializer, group.id=group1, bootstrap.servers=localhost:9092}
res6_1: Object = null
res6_2: Object = null
res6_3: Object = null
res6_4: Object = null
consumer: KafkaConsumer[Long, Array[Byte]] = org.apache.kafka.clients.consumer.KafkaConsumer@31fc239b

In [8]:
case class TransactionRecord(
    eventTime: Timestamp,
    txnID: Long,
    Amount: Double,
    IsFraud: Int,
    V4: Double,
    V9: Double,
    V11: Double,
    V14: Double,
    V16: Double,
    LOCATION_ID: Long,
    ACC_NO: Long,
    IsRedemption: Int=0
){
    override def toString():String = {
        val sb = new StringBuilder("txnID: ")
        sb.append(this.txnID)
        sb.append(", Event time=")
        sb.append(this.eventTime)
        sb.append(", Amount=")
        sb.append(this.Amount)
        sb.append(", IsRedemption? ")
        sb.append(this.IsRedemption)
        sb.append(", IsFraud? ")
        sb.append(this.IsFraud)
        sb.append(", Location=")
        sb.append(this.LOCATION_ID)
        sb.append(", Account No=")
        sb.append(this.ACC_NO)
        sb.append(", V4=")
        sb.append(this.V4)
        sb.append(", V9=")
        sb.append(this.V9)
        sb.append(", V11=")
        sb.append(this.V11)
        sb.append(", V14=")
        sb.append(this.V14)
        sb.append(", V16=")
        sb.append(this.V16)
        return sb.toString
    }
}

defined class TransactionRecord

In [9]:
println("Example of a transaction:")
println(
    new TransactionRecord(
        new java.sql.Timestamp(1610100000000L), 11, 1.1, 0, 0.04, 0.09, 0.11, 0.14, 0.16, 99, 99999, 0
    )
)

Example of a transaction:
txnID: 11, Event time=2021-01-08 15:30:00.0, Amount=1.1, IsRedemption? 0, IsFraud? 0, Location=99, Account No=99999, V4=0.04, V9=0.09, V11=0.11, V14=0.14, V16=0.16


In [10]:
def deserializeTransactionRecord(id:Long, bytes: Array[Byte]): TransactionRecord = {
    
    val buffer = ByteBuffer.wrap(bytes)
    
    val eventTime = new java.sql.Timestamp(buffer.getLong)
    val V4 = buffer.getDouble
    val V9 = buffer.getDouble
    val V11 = buffer.getDouble
    val V14 = buffer.getDouble
    val V16 = buffer.getDouble
    val amount = buffer.getDouble
    val isFraud = buffer.getInt
    val location = buffer.getLong
    val acc_no = buffer.getLong
    val isRedemption = buffer.getInt
    
    TransactionRecord(eventTime, id, amount, isFraud, V4, V9, V11, V14, V16, location, acc_no, isRedemption)
}

defined function deserializeTransactionRecord

In [ ]:
var counter:Int = 0
var prevCount:Int = 0

consumer.subscribe(Seq(topic).asJava)

try {
    while (counter < 50) {
        val records = consumer.poll(java.time.Duration.ofMillis(500))

        for (record <- records.asScala) {
            val txn_id = record.key()
            val valueBytes = record.value()
            val customRecord = deserializeTransactionRecord(txn_id, valueBytes)
            println(customRecord)
            counter = counter + 1
        }
        if (counter > prevCount){
            println(s"*** Received $counter messages till now. ***")
            prevCount = counter
        }
    }
}

	Got record: txnID: 1, Event time=2021-04-04 05:57:24.227, Amount=14962.31, IsRedemption? 0, IsFraud? 0, Location=15, Account No=11, V4=1.378155224, V9=0.36378697, V11=-0.551599533, V14=-0.311169354, V16=-0.470400525
	Got record: txnID: 2, Event time=2021-04-04 05:57:24.227, Amount=269.02, IsRedemption? 0, IsFraud? 0, Location=19, Account No=21, V4=0.448154078, V9=-0.255425128, V11=1.612726661, V14=-0.143772296, V16=0.463917041
	Got record: txnID: 4, Event time=2021-04-04 05:59:04.227, Amount=12350.41, IsRedemption? 0, IsFraud? 0, Location=17, Account No=4, V4=-0.863291275, V9=-1.387024063, V11=-0.226487264, V14=-0.287923745, V16=-1.059647245
	Got record: txnID: 5, Event time=2021-04-04 06:00:44.227, Amount=6999.16, IsRedemption? 0, IsFraud? 0, Location=22, Account No=9, V4=0.403033934, V9=0.817739308, V11=-0.822842878, V14=-1.119669835, V16=-0.451449183
	Got record: txnID: 3, Event time=2021-04-04 05:59:04.227, Amount=37866.68, IsRedemption? 0, IsFraud? 0, Location=13, Account No=44, 

In [ ]:
consumer.close()